# Formulating the Problem

In class we saw an example of an optimisation problem. Analysis of the business problem revealed a suitbale formulation for the problem as a Linear Programming problem. Once the problem could be formulated it could then be written down as a mathematical model. Once the mathematical model is determined, it remains to take this problem and write it down using a modelling language so that we may use computational tools to solve it. This is usually the easiest part of the solution process, and may not be the final step. Inspection of the solution to the problem as formulated may reveal deficiencies in the modelling, necessitating changes to the formulation or even to the problem that is being solved.

In this tutorial we will focus on taking a simple business problem and interpreting it as a linear programming problem. Once we can do this, we will then focus on writing the problem with Mosel and then solving it, as well as inspecting the properties of the solution.

## The Business Problem

Here we have a _product mix_ problem. The essence of it is that SuitUsir, a company that manufactures suits wishes to alter its production strategy to adapt to a changing market. The company has limited resources, however, and although it can produce multiple new lines of suits, it is constrained by production capacity and raw materials. The goal is to find the production mix that gives the maximum profit, given these production capacity constraints. 

As stated in class, to be able to formulate this problem, we need to know some information (which must be gathered by someone, and we hope that it is reliable, otherwise our modelling and optimising may be a futile task) first:

- In which facilities can we produce each type of suit?
- What production capacity is required per unit for each type of suit?
- How much capacity is available at each location?
- How much profit can we make for each type of suit, per unit?

Once we know these, we can start modelling. They are outlined in the table below:

![SuitUsir Production Parameters](images/suitusir.png)

We can see that in Donegal can we only devote additional resources to produce woolen suits. Four percent of the capacity is allocated for production and each unit requires one percent of the *total* capacity. Belfast can only be used to produce linen suits, which require percent per unit of the total industrial capacity. However, in Belfast, we have twelve percent of the toal capacity available. This means that there is the potential to produce as many as six linen suits perhour. In Dublin we can use additional resources to produce both types of suit, with the woolen and linen suits requiring three and two percent of the toal capacity respectively and an available capacity of eighteen percent.

It is clear to see that the linen suits produce more profit per unit, but it remains to see whether or not we can maximise profits by focusing entirely on this line of suits. The solution is subject to the capacity constraints.

## Formulating the Linear Programme

To make decisions about the type of suits that we produce, we need to decide how much of each suit to make. This sounds like a redundant statement, but what it tells us is that we need a decision variable to tell us how many woolen suits we will make and how many linen suits we will make.

To that end, we introduce two variables to $$x_{woollen}$$ and $$x_{linen}$$.

Now, we will operate on a fixed-cost business model for now, which may not be realistic, but is sufficient for us to understand the basics of modelling. We assume that no matter how many suits of each kind we make, the profit will always be the same because the costs do not change, nor does the capacity requirements. 

We will take the variables (for now) to be non-negative continuous variables. That means that each will, in the solution, be a real number telling us what percentage of production to devote to suit productions. This may also be somewhat unrealistic, but we will address this issue later.

$$x_{woollen}, x_{linen} \in \mathbb{R}^{+}_{0}$$

or:

$$x_{woollen}, x_{linen} \geq 0 $$

We can see that at the Donegal plant, we must not allow production to exceed four percent of the total industrial capacity. We may write this constraint as:

$$(1 \times x_{woollen}) + (0 \times x_{linen}) \leq 4$$

Notice that we have unity in front of the first term (since each woollen suit requires one percent of the total capacity there) and a zero in front of the second term because we may not produce any linen suits in Donegal. Next, we take into account the similar constraints for the Belfast facility:

$$(0 \times x_{woollen}) + (2 \times x_{linen}) \leq 12$$

And then for the Dublin facility we can produce both types of suit, so both terms are included:

$$ (3 \times x_{woollen}) + (2 \times x_{linen}) \leq 18 $$

Finally, we note that we wish to maximise the profit we make. This profit, represented by a single scalar (the sum of the profits that we make by producing any number of both types of suit) and is a quantity used to emasure the quality of the solution. Our objective here is to find the solution(s) that maximise this quantity.

If we take it into account, we may write the full linear programme as:

$$maximise \quad \quad (3 \times x_{woollen}) + (5 \times x_{linen})$$ subject to:
$$(1 \times x_{woollen}) + (0 \times x_{linen}) \leq 4$$
$$(0 \times x_{woollen}) + (2 \times x_{linen}) \leq 12$$
$$ (3 \times x_{woollen}) + (2 \times x_{linen}) \leq 18 $$
$$x_{woollen}, x_{linen} \geq 0 $$

We may assume the variables are real, since we are defining a linear programme.

## Writing a Linear Programme with Mosel

Once we have gone through the steps of collecting data, modelling and formulating, only then may we consider attempting to solve the problem. In this case, the problem is small so we may write it in Mosel manually. We can enter the data line-by-line. For large linear programmes, we may not be able to do this, but we need not worry about that yet. 

First, let us declare the variables. First we need only declare that they exist (since we don't know any appropriate initial values for them in general). We will need two variables, one to decide how many woollen suits we should make and one to decide how many linen suits we should make.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen: mpvar
        x_linen: mpvar
    end-declarations
end-model

Next we need to define the objective function.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen: mpvar
        x_linen: mpvar
    end-declarations
    profit := 3 * x_woollen + 5 * x_linen
end-model

Following this, we define each of the constraints:

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen: mpvar
        x_linen: mpvar
    end-declarations
    profit := 3 * x_woollen + 5 * x_linen
    donegal := x_woollen <= 4
    belfast := 2 * x_linen <= 12
    dublin := 3 * x_woollen + 2 * x_linen <= 18
end-model

Finally, we need to tell Mosel that we wish to solve a maxmimisation problem.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen: mpvar
        x_linen: mpvar
    end-declarations
    profit := 3 * x_woollen + 5 * x_linen
    donegal := x_woollen <= 4
    belfast := 2 * x_linen <= 12
    dublin := 3 * x_woollen + 2 * x_linen <= 18
    maximize(profit)
end-model

This will have called Xpress to solve our Mosel model. Once we do this (it should not take long at all), we would like to inspect the solution. Two major things interest us for now: the profit that we make and the values of each variable, that is, the percentage of production we allocate to the suits of each type.

We use the _getobjval_ method to retrieve the objective value of the optimal solution (the product mix that maximise the profit).

We use the _getsol_ method to retrieve the solution values of the optimal solution.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen: mpvar
        x_linen: mpvar
    end-declarations
    profit := 3 * x_woollen + 5 * x_linen
    donegal := x_woollen <= 4
    belfast := 2 * x_linen <= 12
    dublin := 3 * x_woollen + 2 * x_linen <= 18
    maximize(profit)
    writeln("Number of woollen suits to make: ", getsol(x_woollen))
    writeln("Number of linen suits to make: ", getsol(x_linen))
    writeln("Profit attained: ", getobjval)
end-model

## Limitations of the Model

Here we have made some bold assumptions about the problem that has allowed us to model it that way. 

- Firstly, we have allowed the constraints affecting one facility to affect another. It is possible that the prodcution capacity constraints may affect each facility differently.
- Secondly, we assume that we can produce any number of suits. In reality, we can only produce them in discrete quantities. We may wish to model out decision variables as integers.
- Thirdly, we might find that the level of profit that we make for producing each suit depends on the facility that we are considering.

Next, we will try to construct an improved model for our problem. We will allow production of linen and woollen suits at each facility to be indepenent in terms of industrial capacity. That is, while production of linen suits at the Donegal facility may be capped at four percent of the Donegal facility's capacity, the production of linen suits in Dublin is not affected by that constraint. This means that we will we need to knwo not how many linen suits are being made in total, but at each facility. Secondly, for the sake of demonstration, we will allow the values of the decision variables to be only integers. 

The analysis team have conferred with the risk department. The risk department has determined that in this period of high inflation, their risk score may not exceed 2%. Production of suits at each facility incurs a risk score that grows linearly with the number of units produced. Although production of suits at one facility is not impacted by the capacity at one facility, we note that risk is a quantity that is common to the entire production, and wherever the production is, the risk may not exceed the indicated score. See the new table below for the parameters of the new problem.  

![SuitUsir Production Parameters](images/suitUsirExpanded.png)

## The Expanded Model

First, we need to make sure that we are able to decide how many of each suit we produce at each facility. This means that we now need a decision variable for each suit type at each facility. We start the Mosel model by declaring these variables:

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen_donegal: mpvar
        x_linen_donegal: mpvar
        x_woollen_belfast: mpvar
        x_linen_belfast: mpvar
        x_woollen_dublin: mpvar
        x_linen_dublin: mpvar
    end-declarations
end-model

As we said, we wish for these values to be integral, so we must declare that.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen_donegal: mpvar
        x_linen_donegal: mpvar
        x_woollen_belfast: mpvar
        x_linen_belfast: mpvar
        x_woollen_dublin: mpvar
        x_linen_dublin: mpvar
    end-declarations
    x_woollen_donegal is_integer
    x_linen_donegal is_integer
    x_woollen_belfast is_integer
    x_linen_belfast is_integer
    x_woollen_dublin is_integer
    x_linen_dublin is_integer
end-model

Following that, we will define the new profit objective. Since the prices depend not just on the type of suit but also where it is made, we need to have a term in the objective function to reflect that.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen_donegal: mpvar
        x_linen_donegal: mpvar
        x_woollen_belfast: mpvar
        x_linen_belfast: mpvar
        x_woollen_dublin: mpvar
        x_linen_dublin: mpvar
    end-declarations
    x_woollen_donegal is_integer
    x_linen_donegal is_integer
    x_woollen_belfast is_integer
    x_linen_belfast is_integer
    x_woollen_dublin is_integer
    x_linen_dublin is_integer
    profit := (8 * x_woollen_donegal) + (3 * x_linen_donegal) + (4 * x_woollen_belfast) + (6 * x_linen_belfast) + (5 * x_woollen_dublin) + (7 * x_linen_dublin)
end-model

We also need to update the constraints to reflect the fact that the industrial capacities are independent of each other.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen_donegal: mpvar
        x_linen_donegal: mpvar
        x_woollen_belfast: mpvar
        x_linen_belfast: mpvar
        x_woollen_dublin: mpvar
        x_linen_dublin: mpvar
    end-declarations
    x_woollen_donegal is_integer
    x_linen_donegal is_integer
    x_woollen_belfast is_integer
    x_linen_belfast is_integer
    x_woollen_dublin is_integer
    x_linen_dublin is_integer
    profit := (8 * x_woollen_donegal) + (3 * x_linen_donegal) + (4 * x_woollen_belfast) + (6 * x_linen_belfast) + (5 * x_woollen_dublin) + (7 * x_linen_dublin)
    donegal := 2 * x_woollen_donegal +  x_linen_donegal <= 4
    belfast := x_woollen_belfast + 2 * x_linen_belfast <= 12
    dublin := 3 * x_woollen_dublin + 2 * x_linen_dublin <= 18
end-model

We also need to ensure that the risk does not exceed the values we allow for.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen_donegal: mpvar
        x_linen_donegal: mpvar
        x_woollen_belfast: mpvar
        x_linen_belfast: mpvar
        x_woollen_dublin: mpvar
        x_linen_dublin: mpvar
    end-declarations
    x_woollen_donegal is_integer
    x_linen_donegal is_integer
    x_woollen_belfast is_integer
    x_linen_belfast is_integer
    x_woollen_dublin is_integer
    x_linen_dublin is_integer
    profit := (8 * x_woollen_donegal) + (3 * x_linen_donegal) + (4 * x_woollen_belfast) + (6 * x_linen_belfast) + (5 * x_woollen_dublin) + (7 * x_linen_dublin)
    donegal := 2 * x_woollen_donegal +  x_linen_donegal <= 4
    belfast := x_woollen_belfast + 2 * x_linen_belfast <= 12
    dublin := 3 * x_woollen_dublin + 2 * x_linen_dublin <= 18
    risk := (0.5 * x_woollen_donegal) + (0.4 * x_linen_donegal) + (0.3 * x_woollen_belfast) + (0.1 * x_linen_belfast) + (0.3 * x_woollen_dublin) + (0.1 * x_linen_dublin) <= 2
end-model

Lastly, we solve the problem and inspect the solution.

In [ ]:
model "SuitUsir"
    uses "mmxprs"
    declarations
        x_woollen_donegal: mpvar
        x_linen_donegal: mpvar
        x_woollen_belfast: mpvar
        x_linen_belfast: mpvar
        x_woollen_dublin: mpvar
        x_linen_dublin: mpvar
    end-declarations
    x_woollen_donegal is_integer
    x_linen_donegal is_integer
    x_woollen_belfast is_integer
    x_linen_belfast is_integer
    x_woollen_dublin is_integer
    x_linen_dublin is_integer
    profit := (8 * x_woollen_donegal) + (3 * x_linen_donegal) + (4 * x_woollen_belfast) + (6 * x_linen_belfast) + (5 * x_woollen_dublin) + (7 * x_linen_dublin)
    donegal := 2 * x_woollen_donegal +  x_linen_donegal <= 4
    belfast := x_woollen_belfast + 2 * x_linen_belfast <= 12
    dublin := 3 * x_woollen_dublin + 2 * x_linen_dublin <= 18
    risk := (0.5 * x_woollen_donegal) + (0.4 * x_linen_donegal) + (0.3 * x_woollen_belfast) + (0.1 * x_linen_belfast) + (0.3 * x_woollen_dublin) + (0.1 * x_linen_dublin) <= 2
    maximize(profit)
    writeln("Number of woollen suits to make in Donegal : ", getsol(x_woollen_donegal))
    writeln("Number of linen suits to make in Donegal : ", getsol(x_linen_donegal))
    writeln("Number of woollen suits to make in Belfast : ", getsol(x_woollen_belfast))
    writeln("Number of linen suits to make in Belfast : ", getsol(x_linen_belfast))
    writeln("Number of woollen suits to make in Dublin : ", getsol(x_woollen_dublin))
    writeln("Number of linen suits to make in Dublin : ", getsol(x_linen_dublin))
    writeln("Number of linen suits to make: ", getsol(x_linen_dublin))
    writeln("Profit attained: ", getobjval)
end-model